In [3]:
import sys
import networkx as nx # type: ignore
import graphviz
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dowhy import CausalModel
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
sys.path.append('../')
from src.data.make_dataset import load_data, merge_all_datasets
#from src.data.preprocess import handle_missing_values, rolling_mean_customer, rolling_mean_seller, haversine, preprocessing
from src.data.preprocess import HandleMissingValues, ProcessingHelpers, Preprocess


In [4]:
state_to_region = {
    'AC': 'North', 'AP': 'North', 'AM': 'North', 'PA': 'North', 'RO': 'North', 'RR': 'North', 'TO': 'North',
    'AL': 'Northeast', 'BA': 'Northeast', 'CE': 'Northeast', 'MA': 'Northeast', 'PB': 'Northeast', 'PE': 'Northeast', 'PI': 'Northeast', 'RN': 'Northeast', 'SE': 'Northeast',
    'GO': 'Central-West', 'MT': 'Central-West', 'MS': 'Central-West', 'DF': 'Central-West',
    'ES': 'Southeast', 'MG': 'Southeast', 'RJ': 'Southeast', 'SP': 'Southeast',
    'PR': 'South', 'RS': 'South', 'SC': 'South'}

In [5]:
# Loading Data
data_dict = load_data()
olist_customers_df = data_dict['olist_customers_df']
olist_geolocation_df = data_dict['olist_geolocation_df']
olist_order_items_df = data_dict['olist_order_items_df']
olist_order_payments_df = data_dict['olist_order_payments_df']
olist_order_reviews_df = data_dict['olist_order_reviews_df']
olist_orders_df = data_dict['olist_orders_df']
olist_products_df = data_dict['olist_products_df']
olist_sellers_df = data_dict['olist_sellers_df']
product_category_name_translation_df = data_dict['product_category_name_translation_df']
olist_closed_deals_df = data_dict['olist_closed_deals_df']
olist_marketing_qualified_leads_df = data_dict['olist_marketing_qualified_leads_df']

In [6]:
df = merge_all_datasets(olist_customers_df, 
                       olist_geolocation_df,
                       olist_order_items_df,
                       olist_order_payments_df,
                       olist_order_reviews_df, 
                       olist_orders_df,
                       olist_products_df, 
                       olist_sellers_df
                       )

In [7]:
handle_missing = HandleMissingValues()
preprocess_helper = Preprocess()
df = handle_missing.dropping_values(df)
df = handle_missing.geolocation_imputation(df)
df = handle_missing.dropping_columns(df)
df = preprocess_helper.preprocessing(df, state_to_region)
df = preprocess_helper.rolling_mean_process(df)
df_final = preprocess_helper.df_final(df)
df_final.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,review_answer_timestamp,order_item_id,product_id,seller_id,payment_value,...,rainfall,product_weight_kg,product_size,no_photos,product_price,is_delivery_late,customer_experience,seller_avg_rating,freight_value,distance_km
0,d455a8cb295653b55abda06d434ab492,944b72539d7e1f7f7fc6e46639ef1fe3,delivered,2017-09-26 22:17:05,2017-09-27 22:24:16,2017-10-11 21:42:45,1.0,a2ff5a97bf95719e38ea2e3b4105bce8,0015a82c2db000af6aaaf3ae2ecb0532,916.02,...,South,11.800,61920.0,2.0,895.0,0,5.0,5.000000,21.02,687.434185
2,9dc8d1a6f16f1b89874c29c9d8d30447,d9442164acf4b03109425633efaa0cfc,delivered,2017-10-12 13:33:22,2017-10-12 13:49:22,2017-10-25 23:15:15,1.0,a2ff5a97bf95719e38ea2e3b4105bce8,0015a82c2db000af6aaaf3ae2ecb0532,916.02,...,Southeast,11.800,61920.0,2.0,895.0,0,5.0,5.000000,21.02,714.295670
1,7f39ba4c9052be115350065d07583cac,d7fc82cbeafea77bd0a8fbbf6296e387,delivered,2017-10-18 08:16:34,2017-10-18 23:56:20,2017-10-28 14:32:42,1.0,a2ff5a97bf95719e38ea2e3b4105bce8,0015a82c2db000af6aaaf3ae2ecb0532,916.02,...,Southeast,11.800,61920.0,2.0,895.0,0,1.0,3.666667,21.02,417.139650
26,eb188a175542057d90b3ca5628b7b5a0,1eda56b47988d592fec945da9ee88c56,delivered,2017-02-18 23:26:24,2017-02-19 01:02:43,2017-03-04 00:45:28,1.0,e251ebd2858be1aa7d9b2087a6992580,001cca7ae9ae17fb1caed9dfb1094831,573.85,...,Southeast,8.875,24080.0,4.0,99.9,0,4.0,4.000000,14.87,523.615247
27,eb188a175542057d90b3ca5628b7b5a0,1eda56b47988d592fec945da9ee88c56,delivered,2017-02-18 23:26:24,2017-02-19 01:02:43,2017-03-04 00:45:28,2.0,e251ebd2858be1aa7d9b2087a6992580,001cca7ae9ae17fb1caed9dfb1094831,573.85,...,Southeast,8.875,24080.0,4.0,99.9,0,4.0,4.000000,14.87,523.615247
